# Fit emimTF2N data

EoS: PR

Parameter T dependence: Order 3 Polynomial

N (total fitting parameters): 8

**this needs work with the initialization

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import polynomial

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/PR_polyTdep_LHS_fromsc.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/PR_polyTdep_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results3 = pd.read_csv('Data/Init_Final/LHS_Output/PR_polyTdep_LHS_fromquad_final.csv',index_col=0)
lhs_results3 = lhs_results3[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

lhs_results4 = pd.read_csv('Data/Init_Final/LHS_Output/PR_polyTdep_LHS_final.csv',index_col=0)
lhs_results4 = lhs_results4[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','sc_param7','sc_param8','SSR']]

In [4]:
lhs_results = pd.concat([lhs_results1,lhs_results2,lhs_results3,lhs_results4],axis=0).reset_index(drop=True)

In [5]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
202,-1.864907,-0.918542,-1.823518,-1.027595,1.384594,1.873063,0.568903,0.459505,13.134518
2773,-1.539544,-1.773441,-0.314784,1.267149,-1.013635,-0.593613,1.340282,1.174336,10.985124
5650,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,-0.180206,-0.034281,1820.684066
6399,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,-0.055488,-0.008857,1625.820619
8905,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,0.149753,0.031776,2287.641900
9613,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,0.082816,0.014809,11.303835
12773,-1.539544,-1.773441,-0.314784,1.267149,-1.013635,-0.593613,1.340282,1.174336,10.985124


In [6]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,sc_param7,sc_param8,SSR
2773,-1.539544,-1.773441,-0.314784,1.267149,-1.013635,-0.593613,1.340282,1.174336,10.985124
12773,-1.539544,-1.773441,-0.314784,1.267149,-1.013635,-0.593613,1.340282,1.174336,10.985124


In [7]:
sc_param1 = lhs_results.at[2773, 'sc_param1']
sc_param2 = lhs_results.at[2773, 'sc_param2']
sc_param3 = lhs_results.at[2773, 'sc_param3']
sc_param4 = lhs_results.at[2773, 'sc_param4']
sc_param5 = lhs_results.at[2773, 'sc_param5']
sc_param6 = lhs_results.at[2773, 'sc_param6']
sc_param7 = lhs_results.at[2773, 'sc_param7']
sc_param8 = lhs_results.at[2773, 'sc_param8']

In [8]:
from hfc125_emimtf2n_PR_polynomial import configuration 

In [9]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [10]:
parameters, obj_value, a = polynomial(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press = 399300, init_x_c1 = 0.2, init_x_c2 = 0.8,
        init_kappa_1_2A = sc_param2, init_kappa_2_1A = sc_param1,
        init_kappa_1_2B = sc_param4, init_kappa_2_1B = sc_param3,
        init_kappa_1_2C = sc_param6, init_kappa_2_1C = sc_param5,
        init_kappa_1_2D = sc_param8, init_kappa_2_1D = sc_param7,
        eps = 0.1, scaling_fac = 1e-7, 
        filename='Data/Fits/Ipopt_Output/PR_polyTdep_final.txt')

2022-09-19 12:00:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:00:35 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [11]:
print('Objective (SSE):',obj_value)

Objective (SSE): 10.985124370807563


In [12]:
print('Parameters:',parameters)

Parameters: fs.properties.PR_kappa_A[emimTf2N,R125]    -4.052504
fs.properties.PR_kappa_A[R125,emimTf2N]    -2.668926
fs.properties.PR_kappa_B[emimTf2N,R125]    10.234131
fs.properties.PR_kappa_B[R125,emimTf2N]     7.533831
fs.properties.PR_kappa_C[emimTf2N,R125]    -8.477582
fs.properties.PR_kappa_C[R125,emimTf2N]    -7.139190
fs.properties.PR_kappa_D[emimTf2N,R125]     2.239885
fs.properties.PR_kappa_D[R125,emimTf2N]     2.255504
dtype: float64


In [13]:
np.savetxt('Data/Fits/PR_params_polyTdep.csv',parameters,delimiter=',')

In [14]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[  39.34734349   18.67755304  -92.85413249  -52.25561578   68.34058027
    48.53094459  -14.9059854   -14.96171425]
 [  18.67755304   10.62079612  -45.51454069  -29.79823288   35.38870102
    27.76624633   -8.58398749   -8.59348769]
 [ -92.85413248  -45.51454069  220.38222305  127.45268095 -163.77776538
  -118.4836607    36.41721405   36.56669275]
 [ -52.25561578  -29.79823288  127.45268096   83.62973019  -99.21398133
   -77.95148794   24.10693009   24.13309536]
 [  68.34058027   35.38870102 -163.7777654   -99.21398133  123.72859054
    92.354352    -28.41204574  -28.54514115]
 [  48.53094458   27.76624633 -118.48366071  -77.95148794   92.35435199
    72.68193822  -22.48490805  -22.50887777]
 [ -14.9059854    -8.58398749   36.41721405   24.10693009  -28.41204574
   -22.48490805    6.92638708    6.96572438]
 [ -14.96171425   -8.59348769   36.56669276   24.13309536  -28.54514115
   -22.50887777    6.96572438    6.97302849]]


In [15]:
np.savetxt('Data/Covariance/PR_Cov_polyTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.5)

m.fs.properties.PR_kappa_A['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_A[R125,emimTf2N]']
m.fs.properties.PR_kappa_A['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_A[emimTf2N,R125]']
m.fs.properties.PR_kappa_B['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_B[R125,emimTf2N]']
m.fs.properties.PR_kappa_B['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_B[emimTf2N,R125]']
m.fs.properties.PR_kappa_C['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_C[R125,emimTf2N]']
m.fs.properties.PR_kappa_C['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_C[emimTf2N,R125]']
m.fs.properties.PR_kappa_D['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_D[R125,emimTf2N]']
m.fs.properties.PR_kappa_D['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_D[emimTf2N,R125]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_emimTf2N"].iloc[i])
    m.fs.liq = data_extended["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_extended["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_extended["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/PR_FittedData_polyTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/PR_vals_polyTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_vecs_polyTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/PR_FIMtrdet_polyTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/PR_FIMvals_polyTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_FIMvecs_polyTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value*1E-2/N
#number of parameters
k = 8

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/PR_AIC_polyTdep.csv')